In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import roc_auc_score

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers
import model_paths

from tensorboardX import SummaryWriter
%load_ext line_profiler

import torch
import model_paths
import model_params as params
import utils.eval as ev
import pandas as pd
import utils.traintest as tt

import argparse


In [2]:
def get_auroc(model_list, model_params, stats, device):
    auroc = []
    success_rate = []
    conf_list = []
    for i, model in enumerate(model_list):
        with torch.no_grad():
            conf = []
            for data, _ in model_params.test_loader:
                data = data.to(device)

                output = model(data).max(1)[0].exp()

                conf.append(output.cpu())

        conf = torch.cat(conf, 0)

        y_true = torch.cat([torch.ones_like(conf.cpu()), 
                            torch.zeros_like(stats[i])]).cpu().numpy()
        y_scores = torch.cat([conf.cpu(), 
                              stats[i]]).cpu().numpy()
        success_rate.append((stats[i] > conf.mean()).float().mean().item())
        auroc.append(roc_auc_score(y_true, y_scores))
        conf_list.append(conf)
    return auroc, success_rate, conf_list

In [3]:
steps = 5
alpha = 3.
restarts = 3
batches = 3
batch_size = 10

In [4]:
device = torch.device('cuda:1')
datasets = ['MNIST', 'FMNIST']

auroc_vec = []
mmc_vec = []
success_rate_vec = []

for dataset in datasets:
    model_params = params.params_dict[dataset]()
    model_path = model_paths.model_dict[dataset]() 
    model_list = [torch.load(file).to(device) for file in model_path.files]
    
    gmm = model_list[-1].mm

    shape = enumerate(model_params.cali_loader).__next__()[1][0][0].shape
    
    stats, bounds = ev.aggregate_adv_stats(model_list, gmm, device, 
                                           shape, classes=model_params.classes, 
                                           batches=batches, batch_size=batch_size, 
                                           steps=steps, 
                                           restarts=restarts, alpha=alpha)
    auroc, success_rate, conf = get_auroc(model_list, model_params, stats, device)
    
    auroc_vec.append(auroc)
    success_rate_vec.append(success_rate)
    mmc_vec.append([c.mean() for c in conf])

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

In [5]:
stats = torch.stack([torch.tensor(mmc_vec),
            torch.tensor(success_rate_vec),
            torch.tensor(auroc_vec)], 2).transpose(0,1)

df_list = []

for i in range(len(model_list)):
    df = pd.DataFrame(stats[i].numpy() )
    df.insert(0, 'A', pd.Series(datasets))
    df.columns = ['DataSet', 'MMC', 'SR', 'AUROC']
    df_list.append(df.set_index('DataSet'))
    
df = pd.concat(df_list, axis=1, keys=model_path.keys)

In [ ]:
df.to_csv('results/attack_stats.csv')

file = open('results/attack_stats.txt','w') 
file.write(df.round(3).to_latex())
file.close() 

In [7]:
df.round(3)

Base                 CEDA               ODIN                  GMM  \
           MMC     SR  AUROC    MMC   SR  AUROC    MMC     SR  AUROC    MMC   
DataSet                                                                       
MNIST    0.992  0.967  0.456  0.992  0.5  0.823  0.101  0.267  0.688  0.992   
FMNIST   0.983  0.033  0.976  0.982  0.0  1.000  0.422  0.000  0.995  0.981   

                    
          SR AUROC  
DataSet             
MNIST    0.0   1.0  
FMNIST   0.0   1.0